# Solving Linear Programs (LPs) with CVXPY and Gurobi

Recall that linear programming is the minimization (or maximization) of an affine objective subject to affine constraints. In this notebook we will solve LPs using the modelling language [CVXPY](https://www.cvxpy.org/) and solver software [Gurobi](https://www.gurobi.com/). Let us begin by solving the toy example of production planning from the lecture slides.

# Production Planning 

The Winnipeg plant of the Gemstone Tool Company (GTC) produces wrenches and pliers. Both tools are made from steel, and the production requires (i) molding the tools on a molding machine and (ii) assembling the tools on an assembly machine.

GTC wants to determine the daily production of wrenches and pliers so as to maximize profits, subject to the daily capacity constraints and demand limits. We have seen that GTC's production planning problem can be formulated as the LP

\begin{array}{clcl}
\displaystyle \mathop{\text{maximize}}_{{\color{blue} x_1, \color{blue} x_2}}   & 0.13\,{\color{blue}x_1}  +  0.1 \,{\color{blue}x_2} \\
\displaystyle \text{subject to} 
& 1.5 \,{\color{blue}x_1} + {\color{blue}x_2}  \leq  27K  \\
& {\color{blue}x_1} + {\color{blue}x_2}  \leq  21K  \\
& 0.3 \,{\color{blue}x_1} + 0.5 \,{\color{blue}x_2}  \leq  9K   &\\
& {\color{blue}x_1}\leq  15K ,\;\; {\color{blue}x_2}   \leq  16K \\
& {\color{blue}x_1}\geq  0 ,\;\; {\color{blue}x_2}   \geq  0.
\end{array}

In the following we solve this problem to find GTC's optimal production plan.

We first install CVXPY and Gurobi. A pip installation of Gurobi automatically includes a limited version, which can solve LPs with 2K variables and 2K constraints. Note that the standard Microsoft Excel Solver has a limit of 200 decision variables for LPs.

In [1]:
import panel as pn
import manganite as mnn
%load_ext manganite

In [2]:
inputs, optimize, results, optimizer_done = mnn.init()

We import CVXPY and Gurobi as follows. We will also need numpy.

In [3]:
import cvxpy as cp
import numpy as np

# Supplier Sourcing

We will now formulate and solve a supplier sourcing problem.

New Bedford Steel (NBS) procures coking coal to produce steel. For next year’s
production, NBS has solicited the following bids from eight different coal mines:

\begin{array}{c | cccccccc}
\hline
\hline
 & Ashley & Bedfort & Consol &Dunby &Earlam &Florence &Gaston &Hopt \\
\hline
Price (\$/mt) & 49,500 & 50,000 & 61,000 &63,500 &66,500 &71,000 &72,500 &80,000  \\
Union? & yes & yes & no & yes &no &yes &no &no \\
Transport & rail & truck & rail &truck &truck &truck &rail &rail \\
Volatility (percentage) & 15 & 16 & 18 & 20 &21 &22 &23 &25 \\
Capacity (mt/yr) &300 &600 &510 &655 &575 &680 &450 &490 \\
\hline
\hline
\end{array}

NBS wants to procure 1,225mt of coking coal with an average volatility of at least 19%. To avert adverse labour relations, at least 50% of the coal should come from union mines. Moreover, at most 650mt (720mt) can be transported via rail (trucks).

*   How much should NBS procure from each mine so as to minimize total costs?
*   What are the total costs? What are the average costs per mt?



## Formulate an LP to solve NBS' supplier sourcing problem



**Exercise 3:**   Identify the decision variables: What decision(s) must be made in order to solve the problem?

  *   $\color{blue} x_1$: amount of coal purchased from Ashley (in mt)
  *   $\color{blue} x_2$: amount of coal purchased from Bedfort (in mt)
  *   $\color{blue} x_3$: amount of coal purchased from Consol (in mt)
  *   $\color{blue} x_4$: amount of coal purchased from Dunby (in mt)
  *   $\color{blue} x_5$: amount of coal purchased from Earlam (in mt)
  *   $\color{blue} x_6$: amount of coal purchased from Florence (in mt)
  *   $\color{blue} x_7$: amount of coal purchased from Gaston (in mt)
  *   $\color{blue} x_8$: amount of coal purchased from Hopt (in mt)

  Denote by $j$ the $j^\text{th}$ supplier, e.g., the $3^\text{rd}$ supplier is Consol. We can thus denote by $\color{blue} x_j$ the amount of coal purchased from supplier $j$.

**Exercise 4:**   Identify the objective function: Typically either “maximize gain”, e.g., profits, or “minimize loss”, e.g., costs

  For notational convenience, denote by $p_j$ the price (\$/mt) of purchasing coal from supplier $j$.

  The objective is to minimize the total cost of purchasing coal: 
  \begin{array}{l@{\quad}l}
\displaystyle \mathop{\text{minimize}}_{\color{blue} {x}}  & \displaystyle \sum_{j=1}^8 p_j  \color{blue} x_j
\end{array}

**Exercise 5:**   Identify the constraints: What hinders me from making infinite profits or incurring zero costs?

  *  We need to procure 1,225mt of coking coal: $\sum_{j=1}^8 \color{blue} x_j = 1,225$

  *  For ease of notation, denote by $v_j$ the volatility of supplier $j$'s coal. We require an average volatility of at least 19%: 
$$\frac{\sum_{j=1}^8 v_j \color{blue} x_j }{\sum_{j=1}^8 \color{blue} x_j} \geq 19$$
The inequality above can be reformulated as the affine constraint $\sum_{j=1}^8 v_j \color{blue} x_j \geq 19 {\sum_{j=1}^8 \color{blue} x_j}$

  * Let us introduce a binary parameter $u_j$ that is equal to 1 if supplier $j$ is a union mine and zero otherwise. At least 50% of the coal should come from union mines: ${\sum_{j=1}^8 u_j \color{blue} x_j} \geq \frac{1}{2} {\sum_{j=1}^8 \color{blue} x_j}$

  * Let us introduce another binary parameter $t_j$ that is equal to 1 if the coal from supplier $j$ is transported via rail and zero otherwise. At most 650mt (720mt) can be transported via rail (trucks):
  $${\sum_{j=1}^8 t_j \color{blue} x_j} \leq 650, \;\; {\sum_{j=1}^8 (1 - t_j) \color{blue} x_j} \leq 720$$

  * Denote by $c_j$ the capacity of supplier $j$. Each coal mine has capacity restrictions: $\color{blue} x_j \leq c_j$ for all $j=1, \dots, 8$

  * We cannot purchase negative amounts: $\color{blue} x_j \geq 0$ for all $j=1, \dots, 8$
















## Solve the formulated LP

We read the supplier data (prices, e.t.c.) from the Excel file SupplierSourcing.xlsx (available on Google classroom). We will implement the LP in a way that would enable us to incorporate additional suppliers (if necessary) without much effort. 

In [4]:
import pandas as pd
import plotly.express as px

In [5]:
#%%cave depends-on supplier_df
#supplier_df = pd.read_excel('SupplierSourcing.xlsx')

In [6]:
supplier_df = pn.widgets.Tabulator(
  value=pd.read_excel('SupplierSourcing.xlsx',index_col=0),
  layout='fit_data_stretch'
)

In [7]:
inputs[0, :2] = pn.Column('## Suppliers', supplier_df)

In [8]:
%%autoupdate cap_chart --depends-on supplier_df --stage inputs -p 1 0 
cap_chart = px.bar(
    data_frame = (supplier_df.value
                  .reset_index()
                  .melt(id_vars="index")
                  .loc[lambda x:x["index"]=="Capacity (mt/yr)"]),
    x="variable", 
    y="value", 
    labels={"variable":"Supplier","value":"Capacity"}, 
    template="plotly_white"
)

In [9]:
supplierdata = supplier_df.value.to_numpy() #dataframe to numpy array

In [10]:
#supplierdata = supplierdata[:,1:] #remove column zero

Define the parameters:

In [11]:
n = len(supplierdata[0]) #number of suppliers
p = (supplierdata[0,:]) #prices
u = (supplierdata[1,:]) #whether from union
t = (supplierdata[2,:]) #whether rail transportation
v = (supplierdata[3,:]) #volatility
c = (supplierdata[4,:]) #capacities

**Exercise 6:** Declare the (column) vector of decision variables.

In [12]:
x = cp.Variable(n)

**Exercise 7:** Form the objective function.

In [13]:
objective = cp.Minimize(p@x)

In [14]:
#Alternative (but less efficient) Code
objfunc = 0
for j in range(n):
    objfunc = objfunc + p[j]*x[j]
objective = cp.Minimize(objfunc)

**Exercise 8:** Implement the constraints.

In [15]:
constraints = [cp.sum(x) == 1225]
constraints.append(v@x >= (0.19)*cp.sum(x))
constraints.append(u@x >= (1/2)*cp.sum(x))
constraints.append(t@x <= 650)
constraints.append((1-t)@x <= 720)
constraints.append(x[:] <= c[:])
constraints.append(x >= 0)

In [16]:
prob = cp.Problem(objective, constraints)

In [17]:
def model(*args):
    prob.solve(solver=cp.GUROBI)

mnn.on_optimize(model)

In [18]:
#prob.solve(solver=cp.GUROBI)



#print("status:", prob.status)

In [19]:
%%autoupdate sol_chart --depends-on supplier_df optimizer_done --stage results -p 0 0 
sol_chart = px.bar(data_frame=pd.DataFrame(
    {"Suppliers":supplier_df.value.columns,
     "Order volume": x.value}),
       x="Suppliers",
       y="Order volume",
       template="plotly_white")

In [20]:
#print("An optimal solution x is")
#print(x.value)
#print("The optimal value is", prob.value)

In [21]:
#averagecost = objective.value/((cp.sum(x)).value)
#print("The average cost per mt is", averagecost)